### 1. History 데이터 전처리하기
##### SQL Server에서 History Table 데이터를 전처리합니다.

In [1]:
import os
import numpy as np
import pandas as pd
import sys
import json

history = pd.read_csv(r"C:\Users\sony6\Downloads\History_20240912162016.csv",encoding='utf-8-sig', low_memory=False) # 전처리 할 코드의 주소를 넣어주세요.

# 'HData' 열을 문자열로 처리 
history['HData'] = history['HData'].astype(str)

# 'HData'의 각 행에 json.loads를 적용하고, 결과로 나온 딕셔너리로 새로운 DataFrame 생성
expanded_history = history['HData'].apply(json.loads).apply(pd.Series)

# 원본 DataFrame에서 'HData'를 제외하고, 확장된 DataFrame과 병합
history = pd.concat([history.drop(columns=['HData']), expanded_history], axis=1)

# 확장된 새로운 DataFrame 출력
history.to_csv(r"C:\Users\sony6\Downloads\History_T115.csv",encoding='utf-8-sig') # 저장할 데이터의 위치를 지정해주세요.

### 2. Event & Error 데이터 전처리하기
##### SQL Server에서 Event, Error Table 데이터를 전처리합니다.

In [4]:
import os
import numpy as np
import pandas as pd
import sys
import json

e_data = pd.read_csv(r"C:\Users\sony6\Downloads\Error_20240912164344.csv", encoding='utf-8-sig')

# 'EData' 컬럼을 먼저 JSON으로 변환
e_data['EData'] = e_data['EData'].apply(lambda x: json.loads(x.replace("'", "\"")) if isinstance(x, str) else x)

# 'EData' 컬럼을 세로로 확장 (explode)
db_expanded = e_data.explode('EData').reset_index(drop=True)

# 확장된 'EData'를 열로 변환
edata_expanded = pd.json_normalize(db_expanded['EData'])

# 기존 데이터프레임에서 'EData' 제거하고 edata_expanded와 병합
e_data = pd.concat([db_expanded.drop(columns=['EData']), edata_expanded], axis=1)

# 결과 출력
e_data.to_csv(r"C:\Users\sony6\Downloads\Error_T115.csv", encoding='utf-8-sig')

### 3. SN 별로 데이터 저장

In [6]:
import os
import pandas as pd

# 저장할 폴더 경로 설정 (History)
output_dir_history = r'C:\Users\sony6\Downloads\T115\history'

# 폴더가 존재하지 않으면 생성
if not os.path.exists(output_dir_history):
    os.makedirs(output_dir_history)

# 각 SN별로 파일을 저장하는 함수 정의
def save_sn_group(sn, group_df, output_dir):
    file_name = f"{sn}_history.csv"
    file_path = os.path.join(output_dir, file_name)
    group_df.to_csv(file_path, index=False)
    print(f"SN {sn} 데이터 저장 완료: {file_path}")

# SN별로 그룹화 및 저장
sn_list = history['SN'].unique()

for sn in sn_list:
    # 각 SN별 그룹을 가져오기
    group_df = history[history['SN'] == sn]
    save_sn_group(sn, group_df, output_dir_history)

# 저장할 폴더 경로 설정 (Error)
output_dir_error = r'C:\Users\sony6\Downloads\T115\error'

# 폴더가 존재하지 않으면 생성
if not os.path.exists(output_dir_error):
    os.makedirs(output_dir_error)

# 각 SN별로 파일을 저장하는 함수 정의
def save_sn_group_error(sn, group_df, output_dir):
    file_name = f"{sn}_error.csv"
    file_path = os.path.join(output_dir, file_name)
    group_df.to_csv(file_path, index=False)
    print(f"SN {sn} 데이터 저장 완료: {file_path}")

# SN별로 그룹화 및 저장
sn_list_error = e_data['SN'].unique()

for sn in sn_list_error:
    # 각 SN별 그룹을 가져오기
    group_df = e_data[e_data['SN'] == sn]
    save_sn_group_error(sn, group_df, output_dir_error)


SN T001215000150015 데이터 저장 완료: C:\Users\sony6\Downloads\T115\history\T001215000150015_history.csv
SN T001221400520052 데이터 저장 완료: C:\Users\sony6\Downloads\T115\history\T001221400520052_history.csv
SN T001221400530053 데이터 저장 완료: C:\Users\sony6\Downloads\T115\history\T001221400530053_history.csv
SN T001223700060006 데이터 저장 완료: C:\Users\sony6\Downloads\T115\history\T001223700060006_history.csv
SN T001223700110011 데이터 저장 완료: C:\Users\sony6\Downloads\T115\history\T001223700110011_history.csv
SN T001215000190019 데이터 저장 완료: C:\Users\sony6\Downloads\T115\history\T001215000190019_history.csv
SN T001215101540154 데이터 저장 완료: C:\Users\sony6\Downloads\T115\history\T001215101540154_history.csv
SN T001215101770177 데이터 저장 완료: C:\Users\sony6\Downloads\T115\history\T001215101770177_history.csv
SN T001223701060106 데이터 저장 완료: C:\Users\sony6\Downloads\T115\history\T001223701060106_history.csv
SN T001223701610161 데이터 저장 완료: C:\Users\sony6\Downloads\T115\history\T001223701610161_history.csv
SN T001223701900190 

### History & Error(Event) 퓨징

In [8]:
import pandas as pd
import os

# 파일 경로 설정
history_dir = r"C:\Users\sony6\Downloads\T115\history"
error_dir = r"C:\Users\sony6\Downloads\T115\error"
merge_dir = r"C:\Users\sony6\Downloads\T115\merge"

# 폴더가 존재하지 않으면 생성
if not os.path.exists(merge_dir):
    os.makedirs(merge_dir)

# history와 error 디렉토리에서 파일 목록 가져오기
history_files = [f for f in os.listdir(history_dir) if f.endswith('.csv')]
error_files = [f for f in os.listdir(error_dir) if f.endswith('.csv')]

# 파일 이름을 기반으로 파일 병합
for file_name in history_files:
    # 파일명 추출 (split 기준 0번째 값)
    base_name = file_name.split('_')[0]

    # 경로 설정
    history_file_path = os.path.join(history_dir, file_name)
    error_file_path = os.path.join(error_dir, f"{base_name}_error.csv")
    
    if not os.path.exists(error_file_path):
        print(f"Error file for {file_name} not found.")
        continue

    # 파일 읽기
    try:
        history_df = pd.read_csv(history_file_path, encoding='utf-8', low_memory=False)
    except UnicodeDecodeError:
        history_df = pd.read_csv(history_file_path, encoding='euc-kr', low_memory=False)

    try:
        error_df = pd.read_csv(error_file_path, encoding='utf-8', low_memory=False)
    except UnicodeDecodeError:
        error_df = pd.read_csv(error_file_path, encoding='euc-kr', low_memory=False)

    # 데이터 병합
    combined_df = pd.concat([history_df, error_df], axis=0, ignore_index=True)

    # TTime 칼럼 기준으로 정렬
    if 'TTime' in combined_df.columns:
        combined_df = combined_df.sort_values(by='TTime')
    else:
        print(f"TTime 칼럼이 {file_name} 데이터프레임에 없습니다.")

    # TTime 칼럼의 ':' 기준 0번째 값을 추출하여 새로운 'TTime_group' 칼럼 생성
    combined_df['TTime_group'] = combined_df['TTime'].apply(lambda x: x.split(':')[0])

    # 'ECount' 칼럼이 비어있는지 확인하여 'Target' 칼럼 생성
    combined_df['Target'] = combined_df.groupby('TTime_group')['ECount'].transform(lambda x: 1 if x.notna().any() else 0)

    # 'Target' 칼럼이 1로 채워진 경우 'ECount' 칼럼에 값이 없는 행에만 1을 할당
    combined_df['Target'] = combined_df.apply(lambda row: 1 if pd.isna(row['ECount']) and row['Target'] == 1 else 0, axis=1)

    # 'E_RPM' 칼럼이 비어있는 경우를 가진 행 제거
    if 'E_RPM' in combined_df.columns:
        combined_df = combined_df.dropna(subset=['E_RPM'])

    # 불필요한 칼럼 제거
    columns_to_remove = ['A2', 'DBV', 'ECount', 'MN', 'MODEL', 'PT', 'T2', 'dtTTime', 'State', 'ERROR_TYPE', 'PN', 'PV', 'DCH', 'DV']
    combined_df = combined_df.drop(columns=[col for col in columns_to_remove if col in combined_df.columns])

    # 파일 저장
    merged_file_path = os.path.join(merge_dir, f"{base_name}.csv")
    try:
        combined_df.to_csv(merged_file_path, index=False, encoding='utf-8')
        print(f"병합된 데이터가 '{merged_file_path}'에 저장되었습니다.")
    except PermissionError:
        print(f"파일 저장 중 권한 오류가 발생했습니다. 파일 '{merged_file_path}'가 열려 있는지 확인하세요.")
    except Exception as e:
        print(f"파일 저장 중 오류가 발생했습니다: {e}")

병합된 데이터가 'C:\Users\sony6\Downloads\T115\merge\T001210400190019.csv'에 저장되었습니다.
병합된 데이터가 'C:\Users\sony6\Downloads\T115\merge\T001215000130013.csv'에 저장되었습니다.
병합된 데이터가 'C:\Users\sony6\Downloads\T115\merge\T001215000150015.csv'에 저장되었습니다.
병합된 데이터가 'C:\Users\sony6\Downloads\T115\merge\T001215000190019.csv'에 저장되었습니다.
병합된 데이터가 'C:\Users\sony6\Downloads\T115\merge\T001215101540154.csv'에 저장되었습니다.
병합된 데이터가 'C:\Users\sony6\Downloads\T115\merge\T001215101770177.csv'에 저장되었습니다.
병합된 데이터가 'C:\Users\sony6\Downloads\T115\merge\T001215101950195.csv'에 저장되었습니다.
병합된 데이터가 'C:\Users\sony6\Downloads\T115\merge\T001220900200020.csv'에 저장되었습니다.
병합된 데이터가 'C:\Users\sony6\Downloads\T115\merge\T001221400520052.csv'에 저장되었습니다.
병합된 데이터가 'C:\Users\sony6\Downloads\T115\merge\T001221400530053.csv'에 저장되었습니다.
병합된 데이터가 'C:\Users\sony6\Downloads\T115\merge\T001223700060006.csv'에 저장되었습니다.
병합된 데이터가 'C:\Users\sony6\Downloads\T115\merge\T001223700110011.csv'에 저장되었습니다.
병합된 데이터가 'C:\Users\sony6\Downloads\T115\merge\T001223701060106.c

### 하나의 파일로 합치기

In [11]:
import os
import pandas as pd

# 데이터 파일이 위치한 디렉토리와 저장할 디렉토리 설정
input_dir = r"C:\Users\sony6\Downloads\T115\merge"
output_dir = r"C:\Users\sony6\Downloads\T115\DB"
output_file = r"C:\Users\sony6\Downloads\T115\DB\최종.csv"

# 결과를 CSV 파일로 저장
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 입력 디렉토리의 모든 CSV 파일을 읽어서 하나의 DataFrame으로 결합
all_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]
combined_df = pd.concat([pd.read_csv(os.path.join(input_dir, file), encoding='euc-kr') for file in all_files], ignore_index=True)

combined_df.to_csv(output_file, index=False, encoding='euc-kr')

print(f"파일이 성공적으로 저장되었습니다: {output_file}")


C:\Users\sony6\AppData\Local\Temp\ipykernel_16504\4129947951.py:15: DtypeWarning: Columns (2,39) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.concat([pd.read_csv(os.path.join(input_dir, file), encoding='euc-kr') for file in all_files], ignore_index=True)


파일이 성공적으로 저장되었습니다: C:\Users\sony6\Downloads\T115\DB\최종.csv
